In [ ]:
import sisl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

First TranSiesta example.

This example will *only* create the structures for input into TranSiesta. I.e. `sisl`'s capabilities of creating geometries with different species is a core functionality which is handy for creating geometries for Siesta/TranSiesta.

This example will teach you one of the *most important* aspects of performing a successful DFT+NEGF calculation. Namely, that the electrodes should *only* couple to its nearest neighbouring cell.

In [ ]:
graphene = sisl.geom.graphene(1.44, orthogonal=True)

In [ ]:
graphene.write('STRUCT_ELEC_SMALL.fdf')
graphene.write('STRUCT_ELEC_SMALL.xyz')

In [ ]:
elec = graphene.tile(2, axis=0)
elec.write('STRUCT_ELEC.fdf')
elec.write('STRUCT_ELEC.xyz')

The above two code blocks writes two different electrodes that we will test in TranSiesta. In this example, we will have the transport direction along the 1st lattice vector (0th index in Python). Note how TranSiesta does not limit your choice of orientation. *Any* direction may be used as a semi-infinite direction, just as in TBtrans.

In [ ]:
device = elec.tile(3, axis=0)
device.write('STRUCT_DEVICE.fdf')
device.write('STRUCT_DEVICE.xyz')

## Exercises

As this is your first example of running TranSiesta there are a few things you should know:

1. *Any* TranSiesta calculation starts by calculating the electrode Hamiltonians and their densities.
  - Start by defining your electrode and figure out how *long* it should be (by the end of this example you should know how)
  - Converge $k$-points and parameters such that the electrode truly reflects your physical boundaries for the bulk electrode.
  - As in [TB 5](../TB_05/run.ipynb) you should *always* use periodicity if allowed. Using Bloch's theorem will greatly increase throughput with little effort.
  - Run the electrode with `siesta` executable as this `siesta --electrode`. This will create the `*.TSHS` file, which is the basic ingredient used in TranSiesta/TBtrans to calculate the self-energies from the given electrode.
  - Additionally, you can read the electrode Hamiltonian (from the `*.TSHS`) into sisl and create band-structures, just as in [TB 1](../TB_01/run.ipynb).
2. Define your device structure by ensuring the electrode length as determined in step 1. 
  - Add a few *screening layers* to ensure a smooth potential towards the bulk electrodes, do this for all electrodes.
  
    (*general advice, you don't need to do it in this example*)
  - Add the scattering part which connects the electrodes.
3. Define the input for TranSiesta, you can with benefit use the bash-script `tselecs.sh` which will generate a default input for $N\ge1$ electrodes and only require slight changes.

   (*general advice: you don't need to do it for this exercise*)
4. Run TranSiesta
  - Go through the output of TranSiesta to assert that it has run without errors (always do this!) (always, **always** do THIS!)
  - Ensure the SCF has indeed converged, TranSiesta should not stop because of the maximum allowed iterations is too small. Optionally, you may use this flag `SCF.MustConverge T` (the default) to make TranSiesta die if it does not converge.
  - Ensure that the `dQ` column is close to $0$, below $0.01$ is preferable. Further, it is advised that the last couple of iterations also obey this condition.
  - Go through the lines after:
  
        ```
        ***************************
        *  WELCOME TO TRANSIESTA  *
        ***************************
        ```
        
    which is the point where TranSiesta starts.  
    Try and familiarize your-self with the output of TranSiesta, lots of information is printed. Some more important than others.
    
5. Run TBtrans. Notice that TBtrans will default to the values given to TranSiesta, so do not be worried about missing `TBT.*` flags in the input files. However, remark that TBtrans will *only* use the `TS.<>` flag if `TBT.<>` does not exist.

### For this example follow these steps:

- Run TranSiesta for the electrode:

       siesta --electrode RUN_ELEC.fdf > RUN_ELEC.out
       
- Run TranSiesta for the device region:

       siesta RUN.fdf > RUN.out

This finalizes your (possibly) first TranSiesta calculation. All should have gone fine here, after some time.
You should also have noticed that TranSiesta is considerably slower than TBtrans. 

- Continue by deleting all `ELEC.*` files
- Edit `RUN_ELEC.fdf` to include the `STRUCT_ELEC_SMALL.fdf` instead of `STRUCT_ELEC.fdf`
- Re-run the electrode calculation (`siesta --electrode RUN_ELEC.fdf > ELEC_SMALL.out`)
- Re-run TranSiesta (`siesta RUN.fdf > TS_SMALL.out`)
- What happens? And more importantly, why?!?

Evidently TranSiesta does not allow too small electrodes as that would create an erroneous coupling along the semi-infinite directions. Luckily, this may easily be inferred in the electrode output by looking for this keyword: `Internal auxiliary supercell`. The output of TranSiesta will print 3 numbers $\langle A\rangle \mathrm{x} \langle B\rangle \mathrm{x}\langle C\rangle$ which corresponds to the number of neighbouring unit-cells the primary unit-cell connects to. The important number to look for is the number corresponding to the semi-infinite direction.  
What should this number be in order to preserve nearest-neighbour unit-cell interactions **only**? Say, if the semi-infinite electrode sits along the $\langle A\rangle$ direction, what should the number $\langle A\rangle$ (always!) be?